In [19]:
import urllib.request, json
data = {'next':"https://declarator.org/api/v1/search/sections/?page=2&person=91"}
years = []
savings = [] 
cars = []
income = []


while data:
    with urllib.request.urlopen(data["next"]) as url:
        data = json.loads(url.read().decode())
    #append
    
return 

None


In [ ]:
import requests
def declarator_generator(response):
    for res in response.json()['results']:
        yield res
    print('Gettings the rest')
    while response.json()['next'] is not None:
        response = requests.get(response.json()['next'])
        for res in response.json()['results']:
            yield res
